---
# Relatório 02 - Análise de Ativos por Setores - Bloomberg Challenge
---

O objetivo desse relatório é identificar ativos a serem analisados e investidos ao longo do Bloomberg Challenge de Outubro/2024, considerando os setores escolhidos pela análise setorial (Relatório 01)

## 1. Importação de bibliotecas e configuração

### 1.1. Importação de Bibliotecas e Configurações de Data

In [8]:
# Importação de dados
import yfinance as yf 

# Análise de dados
import pandas as pd 

# Visualização
import plotly.graph_objects as go 
import plotly.express as px


# Outros
import os 
from datetime import datetime as dt 

data_inicial = dt(2020, 1, 1)
data_final = dt.today()

### 1.2. Importação de lista de ativos

In [3]:
dir = 'data/ativos_selecionados'

ativos_selecionados = pd.DataFrame()
for arq in os.listdir(dir):
    if arq.endswith('.csv'):
        temp = pd.read_csv(dir + '/' + arq)
        ativos_selecionados = pd.concat([ativos_selecionados, temp])

In [5]:
ativos_selecionados.shape

(263, 12)

In [4]:
ativos_selecionados.head()

,Ticker,Nome,Ponderação,Ações,Preço,MainTicker,YFTicker,SetorEconomico,Industria,NomeCompletoParaAuditoria,Pais,Moeda
0,PROT NO Equity,Protector Forsikring ASA,0.001278,50.451,237.50,PROT,PROT,Healthcare,Biotechnology,PROTEONOMIX INC,United States,USD
1,ALAB UW Equity,Astera Labs Inc,0.001276,21.729,52.10,ALAB,ALAB,Technology,Semiconductors,"Astera Labs, Inc.",United States,USD
2,MXL UW Equity,MaxLinear Inc,0.001275,76.115,14.86,MXL,MXL,Technology,Semiconductors,"MaxLinear, Inc",United States,USD
3,HRMY UQ Equity,Harmony Biosciences Holdings Inc,0.001256,29.877,37.29,HRMY,HRMY,Healthcare,Biotechnology,"Harmony Biosciences Holdings, I",United States,USD
4,TVTX UQ Equity,Travere Therapeutics Inc,0.001227,74.851,14.54,TVTX,TVTX,Healthcare,Biotechnology,"Travere Therapeutics, Inc.",United States,USD


#### 1.2.1. Enriquecimento de dados de bolsa

In [28]:
def add_market_information(company_name):
    """ Função para adicionar o país em que o ativo é operado e a moeda utilizada"""
    try:
        ticker = yf.Ticker(company_name)
    except:
        print(company_name)
        raise
    market = ticker.info.get('exchange', 'N/A')

    return market

In [29]:
ativos_selecionados['Bolsa'] = ativos_selecionados['YFTicker'].apply(lambda x: pd.Series(add_market_information(x)))

### 1.3. Importação de Dados OHLCV

In [9]:
dados_ohlcv = {}

for ativo in ativos_selecionados['YFTicker'].unique():
    temp = yf.download(ativo, start=data_inicial.strftime('%Y-%m-%d'), end=data_final.strftime('%Y-%m-%d'), progress=False)
    if not temp.empty:
        dados_ohlcv[ativo] = temp 

len(dados_ohlcv)

230

Como visto acima, 36 ativos não foram importados e, por isso, não estarão sujeitos a essa análise.

## 2. Análise Exploratória dos Ativos

In [10]:
dados_ohlcv.keys()

dict_keys(['PROT', 'ALAB', 'MXL', 'HRMY', 'TVTX', 'KPRX', 'ACT', 'AVIR', 'LBPH', 'INVA', 'YMAB', 'SGH', 'PRAX', 'WOLF', 'COGT', 'NMRA', 'ARQT', 'RLAY', 'AMSF', 'ZYME', 'SPRY', 'MRVI', 'QLGN', 'DAWN', 'IRON', 'CBUS', 'ACET', 'AOSL', 'ELVN', 'VIR', 'RCUS', 'ENTA', 'CGEM', 'ARVN', 'IMNM', 'CKPT', 'SILO', 'DNTH', 'MDXG', 'ERAS', 'PLRX', 'ANAB', 'OLMA', 'AURA', 'MRNA', 'ETNB', 'SRRK', 'ORIC', 'FDMT', 'SMTC', 'IBRX', 'LQDA', 'ADPT', 'ATXS', 'ARCT', 'TNGX', 'SANA', 'RGNX', 'AMBC', 'CVM', 'TYRA', 'HUMA', 'NUVB', 'ACAD', 'SYRS', 'GLSI', 'SAGE', 'ZVRA', 'ALLO', 'SLRN', 'TPST', 'PRTC', 'JSPR', 'DNA', 'CMRX', 'PTGX', 'NVTS', 'LXRX', 'FHTX', 'INAB', 'IGMS', 'MBRX', 'GNPX', 'PRME', 'GYRE', 'AIP', 'SGMO', 'AEON', 'PWI.F', 'BMEA', 'FGEN', 'TSHA', 'SAVA', 'TTNP', 'NVDA', 'AVGO', 'AMD', 'QCOM', 'TXN', 'VRTX', 'ADI', 'MMC', 'REGN', 'MU', 'RELI', 'INTC', 'MRVL', 'AJG', 'MPWR', 'MCHP', 'ALNY', 'ON', 'BRO', 'KROS', 'UTHR', 'FNF', 'SWKS', 'ERIE', 'BMRN', 'INSM', 'TECH', 'PCVX', 'SRPT', 'RPRX', 'INCY', 'ACST'

### 2.1. Obtenção dos Volumes Médios Diários

Um dos critérios possíveis é a escolha de ativos dos setores selecionados que possua volumes suficientes para a modelagem. 
Quanto mais operações, maior a chance de o mercado não ser afetado por um player único.

In [38]:
volumes_medios = {}
for ativo in dados_ohlcv.keys():
    volumes_medios[ativo] = dados_ohlcv[ativo]['Volume'].mean()

volumes_medios = pd.DataFrame(volumes_medios, index = ['Média Volume']).T.sort_values(by='Média Volume', ascending=False)
volumes_medios.reset_index(inplace=True)
volumes_medios.columns = ['YFTicker', 'Média Volume']
volumes_medios.head(20)

,YFTicker,Média Volume
0,NVDA,4.577784e+08
1,AMD,6.507724e+07
2,INTC,3.916639e+07
3,AVGO,2.465012e+07
4,MU,1.973214e+07
5,MRVL,1.047801e+07
6,MRNA,9.476764e+06
7,QCOM,9.137129e+06
8,ON,7.049471e+06
9,NVAX,6.970482e+06


#### 2.1.1. Obtenção de volumes de ativos com indicação de setor

In [46]:
dados_enriquecimento = ativos_selecionados[['YFTicker', 'Industria', 'SetorEconomico', 'NomeCompletoParaAuditoria', 'Bolsa']].drop_duplicates(subset='YFTicker')
vol_med_ref = volumes_medios.merge(dados_enriquecimento, on='YFTicker', how = 'left', validate='one_to_one')
vol_med_ref.head()

,YFTicker,Média Volume,Industria,SetorEconomico,NomeCompletoParaAuditoria,Bolsa
0,NVDA,4.577784e+08,Semiconductors,Technology,NVIDIA Corporation,NMS
1,AMD,6.507724e+07,Semiconductors,Technology,"Advanced Micro Devices, Inc.",NMS
2,INTC,3.916639e+07,Semiconductors,Technology,Intel Corporation,NMS
3,AVGO,2.465012e+07,Semiconductors,Technology,Broadcom Inc.,NMS
4,MU,1.973214e+07,Semiconductors,Technology,"Micron Technology, Inc.",NMS


### 2.2. Distribuição dos Volumes

In [52]:
fig = px.box(vol_med_ref, x="Industria", y="Média Volume", title="Box Plot do Volume Médio por Industria", template="plotly_dark")
fig.show()

Percebe-se que NVIDIA tem sido um outlier em questão de volume

In [53]:
fig = px.box(vol_med_ref, x="Bolsa", y="Média Volume", title="Box Plot do Volume Médio por Bolsa", template="plotly_dark")
fig.show()

O que afeta, inclusive, o comportamento da média de volume da bolsa respectiva (NMS).